<a href="https://colab.research.google.com/github/cerr/pycerr-notebooks/blob/main/autosegment_CT_Lung_OARs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

In this tutorial, we will demonstrate how to apply a pre-trained AI model to segment the OARs on a lung CT scan using pyCERR.  

## Requirements
* Python>=3.8
* Running this model requires access to a GPU.  
  *On Colab* :  `Runtime > Change runtime type > Select GPU `

## AI model
* The segmentation model used here was trained and validated on CT scans used for RT planning. Its performance on diagnostic CTs is expected to be sub-optimal.
* The trained model is packaged as a Conda environment archive containing  python libraries and other dependencies.

## I/O
* **Input**: DICOM-format planning lung CT scan(s).

* **Output**: DICOM RTStruct-format segmentations.

  Input data should be organized as: one directory of DICOM images per patient.    
  
    
  ```center  
    Input dir
            |------Pat1  
                      |------img1.dcm  
                             img2.dcm  
                             ....  
                             ....  
            |-----Pat2  
                     |------img1.dcm  
                            img2.dcm  
                            ....  
                            ....  
    
  ```

## Installing the model and its dependencies

* Installation is performed using CERR's [***model installer***]( https://github.com/cerr/model_installer).

* A Conda archive containing dependencies is downloaded to the `conda-pack`   
  sub-directory of a configurable `scriptInstallDir`.  
  By default `condaEnvPath = '/content/CT_LungOAR_incrMRRN/conda-pack'`

* The inference script is located at   
  `scriptInstallDir = os.path.join(condaEnvPath,'model_wrapper', run_inference_nii.py')`

## Running the model
* Command to execute the model
```python
!python {wrapperPath} {input_nii_directory} {output_nii_directory}
```
* Data I/O, pre- and post-processing are performed using [***pyCERR***](https://github.com/cerr/pyCERR) .

## License

By downloading the software you are agreeing to the following terms and conditions as well as to the Terms of Use of CERR software.

**`THE SOFTWARE IS PROVIDED "AS IS" AND CERR DEVELOPMENT TEAM AND ITS COLLABORATORS DO NOT MAKE ANY WARRANTY, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE, NOR DO THEY ASSUME ANY LIABILITY OR RESPONSIBILITY FOR THE USE OF THIS SOFTWARE.`**

`This software is for research purposes only and has not been approved for clinical use.`

`Software has not been reviewed or approved by the Food and Drug Administration, and is for non-clinical, IRB-approved Research Use Only. In no event shall data or images generated through the use of the Software be used in the provision of patient care.`
  
`YOU MAY NOT DISTRIBUTE COPIES of this software, or copies of software derived from this software, to others outside your organization without specific prior written permission from the CERR development team except where noted for specific software products.`

`All Technology and technical data delivered under this Agreement are subject to US export control laws and may be subject to export or import regulations in other countries. You agree to comply strictly with all such laws and regulations and acknowledge that you have the responsibility to obtain
such licenses to export, re-export, or import as may be required after delivery to you.`

**`You may publish papers and books using results produced using software provided that you reference the following`**:
  
  * **AI model**: https://doi.org/10.1109/TMI.2018.2857800  


# Downloads


### Download planning CTs (DICOM)


The following can be used to download data from user-specified ***dataUrl*** to ***dataDownloadDir***.

See [demo notebook](github.com/cerr/pyCERR-Notebooks/download_data_from_xnat.ipynb) for downloading data from XNAT.

In [ ]:
import os
workDir = r'/content' #for Colab
#workDir = r'/home/jupyter'
dataUrl = 'http://path.to/data'
dataDownloadDir = os.path.join(workDir,'sampleData')
os.makedirs(dataDownloadDir,exist_ok=True)

In [ ]:
%%capture
! wget -O sampleData.gz -L {dataUrl}
! tar xf sampleData.gz -C {dataDownloadDir}
! rm sampleData.gz

Define paths to input DICOM directory, desired output directory, and a session directory to store temporary files during model execution

In [ ]:
#Paths to input data and conda env with pre-trained models

# Replace with appropriate path to your dataset
inputDicomPath = os.path.join(dataDownloadDir,'your_dir_name_here')
# Dir to hold temp files
sessionPath = os.path.join(workDir, 'temp')
# Dir to store AI model outputs
outputDicomPath = os.path.join(workDir, 'AIoutput')

if not os.path.exists(outputDicomPath):
  os.mkdir(outputDicomPath)

if not os.path.exists(sessionPath):
  os.mkdir(sessionPath)

### Download pre-trained model, inference script, and packaged conda evnironment to ***scriptInstallDir***


In [ ]:
%%capture
os.chdir(workDir)
!git clone https://github.com/cerr/model_installer.git
os.chdir(os.path.join(workDir,'model_installer'))
!./installer.sh -h

modelOpt = '2'  # CT_LungOAR_incrMRRN
pythonOpt = 'C' # Download packaged Conda environment

! source ./installer.sh -m {modelOpt} -d {workDir} -p {pythonOpt}

In [ ]:
# Location of inference script
scriptInstallDir = os.path.join(workDir, 'CT_LungOAR_incrMRRN')
scriptPath = os.path.join(scriptInstallDir,
                         'model_wrapper',
                         'run_inference_nii.py')

# Location of Conda archive
condaEnvPath = os.path.join(scriptInstallDir, 'conda-pack')

# Location of activation script for Conda environment
activateScript = os.path.join(condaEnvPath,'bin','activate')

### Install ***pyCERR***

pyCERR is used for data import/export and pre- and post-processing transformations needed for this model.

In [ ]:
%%capture
! pip install "pyCERR[napari] @ git+https://github.com/cerr/pyCERR.git@testing"

# Data processing using pyCERR

### Pre-processing

### `processInputData` Identify the input scan and resize slices to 512x512

In [ ]:
import cerr
from cerr.dataclasses import structure
from cerr.contour import rasterseg as rs
from cerr.utils.image_proc import resizeScanAndMask
from cerr.utils.ai_pipeline import getScanNumFromIdentifier

def processInputData(planC):

  # Identify scan index in  planC
  scanIdS = {"imageType": "CT SCAN"}
  matchScanV = getScanNumFromIdentifier(scanIdS,
                                        planC,
                                        False)

  # Extract scan
  scanNum = matchScanV[0]
  scan3M = planC.scan[scanNum].getScanArray()
  mask3M = None

  # Resize scan and import to planC
  inputImgSizeV = np.shape(scan3M)
  gridS = planC.scan[scanNum].getScanXYZVals()
  outputImgSizeV = [512, 512, inputImgSizeV[2]]
  method = 'padorcrop3d'
  procScan3M, __, resizeGridS = resizeScanAndMask(
                                scan3M, mask3M, gridS,
                                outputImgSizeV, method)

  return procScan3M, resizeGridS


## Post-processing

### `postProcAndImportSeg` Import label maps and retain only the largest connected component to filter out false detections.

In [ ]:
# Map output labels to structure names
strToLabelMap = {1:"Lung_Left", 2:"Lung_Right", 3:"Heart", 4:"Esophagus", \
                 5:"Cord", 6:"PBT"}
numLabel = len(strToLabelMap)

In [ ]:
#Import label map to CERR
import glob
from cerr.utils import mask
from cerr.dataclasses import structure as cerrStr

def postProcAndImportSeg(outputDir,procScanNum,scanNum,planC):
  niiGlob = glob.glob(os.path.join(outputDir,'*.nii.gz'))

  print('Importing ' + niiGlob[0]+'...')
  numStrOrig = len(planC.structure)
  planC = pc.loadNiiStructure(niiGlob[0], procScanNum,
                              planC, \
                              labels_dict = strToLabelMap)

  cpyStrNumV = np.arange(numStrOrig,len(planC.structure))
  numComponents = 1
  for label in range(numLabel):
    # Copy to original scan
    planC = structure.copyToScan(cpyStrNumV[label], scanNum, planC)
    origStr = len(planC.structure)-1
    strName =  strToLabelMap[label+1]
    # Post-process and replace input structure in planC
    procMask3M = cerrStr.getLargestConnComps(origStr, numComponents,
                                             planC, saveFlag=True,
                                             replaceFlag=False,
                                             procSructName=strName)
  return planC

# Segment OARs

## Apply AI model  to all MR scans

### located in ***inputDicomPath*** and store auto-segmentation results to ***outputDicomPath***

In [ ]:
%%capture
import subprocess
import numpy as np

from cerr import plan_container as pc
from cerr.dataclasses import scan as cerrScn
from cerr.utils.ai_pipeline import createSessionDir
from cerr.dcm_export import rtstruct_iod

# Loop over pyCERR files
fileList = os.listdir(inputDicomPath)
numFiles = len(fileList)
modality = 'CT'
scanNum = 0

for iFile in range(numFiles):

    inputFilename = fileList[iFile]
    dcmDir = os.path.join(inputDicomPath,inputFilename)

    # Create session dir to store temporary data
    modInputPath, modOutputPath = createSessionDir(sessionPath,
                                                   inputDicomPath)

    # Import DICOM scan to planC
    planC = pc.loadDcmDir(dcmDir)
    numExistingStructs = len(planC.structure)

    # Pre-process data
    procScan3M, resizeGridS = processInputData(planC)
    planC = pc.importScanArray(procScan3M, resizeGridS[0], \
                               resizeGridS[1], resizeGridS[2],\
                               modality, scanNum, planC)
    procScanNum = len(planC.scan) - 1

    # Export inputs to NIfTI
    scanFilename = os.path.join(modInputPath,
                                f"{inputFilename}_scan_3D.nii.gz")
    planC.scan[procScanNum].saveNii(scanFilename)

    # Apply model
    subprocess.run(f"source {activateScript} && python {scriptPath}\
                   {modInputPath} {modOutputPath}",\
                   capture_output=False, shell=True, executable="/bin/bash")

    # Import results to planC
    planC = postProcAndImportSeg(modOutputPath, procScanNum, scanNum, lanC)
    newNumStructs = len(planC.structure)

    # Export segmentations to DICOM
    structFileName = inputFilename + '_AI_seg.dcm'
    structFilePath = os.path.join(outputDicomPath, structFileName)
    seriesDescription = "AI Generated"
    exportOpts = {'seriesDescription': seriesDescription}

    structNumV = np.arange(numExistingStructs, newNumStructs)
    indOrigV = np.array([cerrScn.getScanNumFromUID(planC.structure[structNum].assocScanUID,\
                        planC) for structNum in structNumV], dtype=int)
    structsToExportV = structNumV[indOrigV == scanNum]

    rtstruct_iod.create(structsToExportV, structFilePath,
                        planC, exportOpts)

## **Optional**: Uncomment the following to download the output segmentations to your workspace bucket.

In [ ]:
# workspaceBucket = os.environ['WORKSPACE_BUCKET']
# !gcloud storage cp -r {outputDicomPath} {workspaceBucket}

# Display results

## Overlay AI segmentations on scan for visualization using ***Matplotlib***

Note: This example displays the last segmented dataset by default.    
Load the appropriate pyCERR archive to `planC` to view results for desired dataset.

In [ ]:
from cerr.viewer import showMplNb

showMplNb(planC=planC, scan_nums=scanNum,
          struct_nums=structsToExportV,
          windowCenter=-400, windowWidth=2000)

Output()